In [51]:
from collections import Counter
import math, glob, csv, re, numpy
from matplotlib import pyplot
import json
from collections import defaultdict
import os
import fnmatch
from datetime import datetime


In [75]:
def find_all(name, path):
    l={}
    for filename in os.listdir(path):
#         print (filename)
        if fnmatch.fnmatch(filename,"*"+name+"*"):
#             print (filename)
#             print (load_album("../"+filename)[0]['release_date'])
            with open("../male_lyrics/"+filename) as json_file:
                data = json.load(json_file)
#                 print (data)
                date = datetime.strptime(data["songs"][0]['year'], '%Y-%M-%d')
                l[filename]=date
#     print (l)
    #     sorted_songs_artist = sorted(l, key=lambda x: x["release_date"])    
    l={k: v for k, v in sorted(l.items(), key=lambda item: item[1])}
  
    sortedl=list(l.keys())
    return sortedl

In [76]:
def counter_to_vector(counter,vocab,reverse):
    vector = numpy.zeros(vocab)
    for word in counter.keys():
        ## look up the integer ID for the string
        word_id = reverse[word]
#         print (counter[word])
        vector[word_id] = counter[word]
    
    return vector / vector.sum()

In [77]:
filenames=find_all("Tyler","../male_lyrics")

In [91]:
def album_kl(artist):
    complete=[]
    filenames=find_all(artist,"../male_lyrics")
    
#     print (type(filenames))
    
    for file in filenames:
        with open("../male_lyrics/"+file) as json_file:
            data = json.load(json_file)
            complete.append(data)
    word_pattern = re.compile("\w[\w\-\']*\w|\w")

    albumlist = defaultdict(list)

# all_counts = Counter()


#     print (len(complete))
    for albums in complete:
        
#         print (albums)
        line_buffer = []
        for row in albums["songs"]:
#             print (row)
#             print (row["album"])
    
# #     for row in data:
        
#         print("Reading {} ({})".format(row["title"], row["album"]))
        
#         if row["album"] not in albumlist:
#             albums[row["album"]]=[]
            if row["album"] in ['Get Well Soon...','Kon the Louis Vuitton Don',"I’m Good"]:
                row['album']="The College Dropout"
                albumlist[row["album"]].append(row)

    # # #         row["Year"] = int(row["Year"])

                counter = Counter()
                row["Tokens"] = counter


    # #         print ((row['lyrics']))
                line_buffer.append(row['lyrics'])
    #         print (line_buffer)
                counter.update(word_pattern.findall(" ".join(line_buffer)))
                line_buffer = []
            else:
                albumlist[row["album"]].append(row)
            
# # #         row["Year"] = int(row["Year"])
        
                counter = Counter()
                row["Tokens"] = counter
        
        
# #         print ((row['lyrics']))
                line_buffer.append(row['lyrics'])
#         print (line_buffer)
                counter.update(word_pattern.findall(" ".join(line_buffer)))
                line_buffer = []
#     all_counts += counter
#     print (albumlist.keys())
    print (albumlist[None])
    albumtokens=defaultdict(dict)
    for album in albumlist:
#     print (type(albumlist[album]))
        albumtokens[album]["Tokens"]=Counter()
        for song in albumlist[album]:
            albumtokens[album]["Tokens"].update(song["Tokens"])
    check=Counter()
    for i in albumtokens:
        check.update(albumtokens[i]["Tokens"])
        
    vocabulary = [w for w, c in check.most_common()]
# print (vocabulary)
    vocabulary_size = len(vocabulary)
#     print (vocabulary_size)
     
    reverse_vocab = { w: i for i, w in enumerate(vocabulary) }
    all_vector = counter_to_vector(check,vocabulary_size,reverse_vocab)
    
#     print (albumtokens[i]["Tokens"])
#     print (check)
    
    
    print (albumtokens.keys())
    for album in albumtokens:
    #     print (albumtokens[album]["Tokens"]["Yo"])
        albumtokens[album]["Vector"] = counter_to_vector(albumtokens[album]["Tokens"],vocabulary_size,reverse_vocab)
    
    for album in (albumtokens):
#     print 
    # `nonzero` returns a tuple with one element per dimension of the input array
        nzs = numpy.nonzero(albumtokens[album]["Vector"])[0]
    #     print (nzs)
        kl_terms = albumtokens[album]["Vector"][nzs] * numpy.log2(albumtokens[album]["Vector"][nzs] / all_vector[nzs])

        vocabulary_subset = [vocabulary[i] for i in nzs]

        sorted_words = sorted(zip(kl_terms, vocabulary_subset), reverse=True)
    #     print(song_id, song["title"])
        print (album)
        print(", ".join(["{} ({:.3f})".format(w, s) for s, w in sorted_words[:40]]))
        print ()
        print(", ".join(["{} ({:.3f})".format(w, s) for s, w in sorted_words[-40:]]))
        print()

In [92]:
album_kl("Childish")

[{'title': 'Secret Track (V. 3005 - Beach Picnic Version)', 'album': None, 'year': '2014-10-05', 'lyrics': 'No matter what you say or what you do\nWhen I\'m alone, I\'d rather be with you\nFuck these other niggas\nI\'ll be right by your side \'til 3005, hold up\n\nYoung God what it is, how you livin\', huh\nYoung thing and she sayin\' let me beat it up\nSee me out in my ride, wanna key it up\nHe got a Kia, I got a Bimmer\nShe got a Fiat, half Italian\nShe wanna ride up, on my stallion\nSee me stylin\', I\'ma ride out\nHit Lihue then we drive to the hide out, uh\nThat\'s the way that we living\nHe fly her out every weekend\nBecause her thinking is different\nShe dance around in the kitchen\nLike "Doo-doo, doo-doo, dun, doo-doo"\nYou\'ll always be my baby\nAsk me what we\'re doing\nYeah, you know that drive me crazy\nShe wanna ride with the top way down\nWhen I get her home I\'ma lay that down\nLay that down, oh-oh\n\nNo matter what you say or what you do\nWhen I\'m alone, I\'d rather be

In [68]:
album_kl("Tyler,")

{'Lyrics_Tyler,TheCreator{.Golbin.json': datetime.datetime(2011, 1, 10, 0, 5), 'Lyrics_Tyler,TheCreator{.Flower-Boy.json': datetime.datetime(2017, 1, 21, 0, 7), 'Lyrics_Tyler,TheCreator{.Igor.json': datetime.datetime(2019, 1, 7, 0, 5), 'Lyrics_Tyler,TheCreator{.Bastard.json': datetime.datetime(2009, 1, 25, 0, 12), 'Lyrics_Tyler,TheCreator{.Cherry-Bomb.json': datetime.datetime(2015, 1, 9, 0, 4), 'Lyrics_Tyler,TheCreator{.Wolf.json': datetime.datetime(2013, 1, 1, 0, 4)}
{'Lyrics_Tyler,TheCreator{.Bastard.json': datetime.datetime(2009, 1, 25, 0, 12), 'Lyrics_Tyler,TheCreator{.Golbin.json': datetime.datetime(2011, 1, 10, 0, 5), 'Lyrics_Tyler,TheCreator{.Wolf.json': datetime.datetime(2013, 1, 1, 0, 4), 'Lyrics_Tyler,TheCreator{.Cherry-Bomb.json': datetime.datetime(2015, 1, 9, 0, 4), 'Lyrics_Tyler,TheCreator{.Flower-Boy.json': datetime.datetime(2017, 1, 21, 0, 7), 'Lyrics_Tyler,TheCreator{.Igor.json': datetime.datetime(2019, 1, 7, 0, 5)}
['Lyrics_Tyler,TheCreator{.Bastard.json', 'Lyrics_Tyle

In [69]:
album_kl("Frank")

{'Lyrics_FrankOcean{.Channel-Orange.json': datetime.datetime(2012, 1, 10, 0, 7), 'Lyrics_FrankOcean{.nostalgia-ULTRA.json': datetime.datetime(2011, 1, 16, 0, 2), 'Lyrics_FrankOcean{.Blonde.json': datetime.datetime(2016, 1, 20, 0, 8), 'Lyrics_FrankOcean{.Endless.json': datetime.datetime(2015, 1, 17, 0, 1)}
{'Lyrics_FrankOcean{.nostalgia-ULTRA.json': datetime.datetime(2011, 1, 16, 0, 2), 'Lyrics_FrankOcean{.Channel-Orange.json': datetime.datetime(2012, 1, 10, 0, 7), 'Lyrics_FrankOcean{.Endless.json': datetime.datetime(2015, 1, 17, 0, 1), 'Lyrics_FrankOcean{.Blonde.json': datetime.datetime(2016, 1, 20, 0, 8)}
['Lyrics_FrankOcean{.nostalgia-ULTRA.json', 'Lyrics_FrankOcean{.Channel-Orange.json', 'Lyrics_FrankOcean{.Endless.json', 'Lyrics_FrankOcean{.Blonde.json']
dict_keys(['nostalgia, ULTRA.', 'channel ORANGE', 'Endless', 'Blonde '])
nostalgia, ULTRA.
can't (0.020), believe (0.018), try (0.018), I (0.017), swim (0.011), numb (0.011), murder (0.010), be (0.009), baby (0.009), novacane (0.00

In [94]:
album_kl("Kendrick")

[]
dict_keys(['Overly Dedicated', 'Section.80', 'good kid, m.A.A.d city (Deluxe Version)', 'To Pimp a Butterfly', 'DAMN.', None])
Overly Dedicated
m (0.007), Patrón (0.006), smoke (0.005), that (0.005), she (0.005), Why (0.005), are (0.005), much (0.004), these (0.004), even (0.004), me (0.004), pussy (0.004), make (0.004), going (0.003), people (0.003), hurt (0.003), Pussy (0.003), don (0.003), Where (0.003), through (0.003), why (0.003), I (0.003), so (0.003), Michael (0.003), niggas (0.003), crack (0.003), O (0.003), t (0.003), like (0.003), dope (0.003), Jordan (0.003), go-go (0.003), H (0.002), something (0.002), too (0.002), So (0.002), that's (0.002), three (0.002), hoes (0.002), we (0.002)

real (-0.001), A (-0.001), not (-0.001), I'll (-0.001), die (-0.001), and (-0.001), his (-0.001), way (-0.001), man (-0.001), When (-0.001), love (-0.001), them (-0.001), If (-0.001), one (-0.001), see (-0.001), never (-0.001), him (-0.001), with (-0.001), he (-0.001), was (-0.001), good (-0

/Users/jeffrey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [95]:
album_kl("Vince")

[]
dict_keys(['Summertime ’06', 'Big Fish Theory', 'FM!', None])
Summertime ’06
okay (0.008), I (0.007), you (0.006), gotta (0.005), it (0.005), lift (0.005), know (0.005), be (0.005), That's (0.005), true (0.005), never (0.004), paid (0.004), nigga (0.004), three (0.004), off (0.004), slang (0.003), roof (0.003), ya (0.003), baby (0.003), and (0.003), jump (0.003), hang (0.003), wanna (0.003), to (0.003), let's (0.003), You (0.003), bang (0.003), could (0.003), keep (0.003), they (0.003), me (0.003), crazy (0.003), at (0.002), for (0.002), Wanna (0.002), need (0.002), how (0.002), if (0.002), So (0.002), wit (0.002)

want (-0.001), bout (-0.001), round (-0.001), When (-0.001), yeah (-0.001), old (-0.001), night (-0.001), from (-0.001), now (-0.001), No (-0.001), If (-0.001), just (-0.001), have (-0.001), by (-0.001), we (-0.001), with (-0.001), hold (-0.001), see (-0.001), won't (-0.001), life (-0.001), Don't (-0.001), was (-0.001), do (-0.001), not (-0.001), way (-0.001), bitch (-0.0

/Users/jeffrey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [96]:
album_kl("A$AP")

[]
dict_keys(['LIVE.LOVE.A$AP', 'LONG.LIVE.A$AP (Deluxe Version)', 'AT.LONG.LAST.A$AP', 'TESTING', None])
LIVE.LOVE.A$AP
get (0.025), lit (0.020), that (0.008), we (0.008), it (0.007), new (0.006), high (0.006), purple (0.005), day (0.005), She's (0.005), is (0.005), these (0.005), bout (0.005), rolling (0.004), brand (0.004), Brand (0.004), roll (0.004), codeine (0.004), sip (0.004), We (0.004), mothafuckin (0.004), promethazine (0.004), I'mma (0.004), bitch (0.003), this (0.003), drinking (0.003), and (0.003), up (0.003), straight (0.003), nigga (0.003), know (0.003), lean (0.003), swag (0.003), in (0.003), Thinking (0.003), Swag (0.003), Hey (0.003), every (0.003), Let (0.003), thoughts (0.003)

It's (-0.001), Tell (-0.001), put (-0.001), My (-0.001), through (-0.001), mind (-0.001), just (-0.001), need (-0.001), could (-0.001), some (-0.001), All (-0.001), time (-0.001), uh (-0.001), or (-0.001), never (-0.001), Uh (-0.001), it's (-0.001), with (-0.001), say (-0.001), at (-0.001), 

/Users/jeffrey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


# end here^

In [56]:
import json
complete=[]
filenames = ["../Lyrics_FrankOcean{.Blonde.json","../Lyrics_FrankOcean{.Channel-Orange.json","../Lyrics_FrankOcean{.Endless.json","../Lyrics_FrankOcean{.nostalgia-ULTRA.json"]
for file in filenames:
    print (file)
    with open(file) as json_file:
        data = json.load(json_file)
        complete.append(data)

../Lyrics_FrankOcean{.Blonde.json


FileNotFoundError: [Errno 2] No such file or directory: '../Lyrics_FrankOcean{.Blonde.json'

In [84]:
word_pattern = re.compile("\w[\w\-\']*\w|\w")

albumlist = defaultdict(list)

# all_counts = Counter()



for albums in complete:
#     print (albums)
    line_buffer = []
    for row in albums["songs"]:
#         print (row["album"])
    
# #     for row in data:
        
#         print("Reading {} ({})".format(row["title"], row["album"]))
        
#         if row["album"] not in albumlist:
#             albums[row["album"]]=[]
        albumlist[row["album"]].append(row)
            
# # #         row["Year"] = int(row["Year"])
        
        counter = Counter()
        row["Tokens"] = counter
        
        
# #         print ((row['lyrics']))
        line_buffer.append(row['lyrics'])
#         print (line_buffer)
        counter.update(word_pattern.findall(" ".join(line_buffer)))
        line_buffer = []
#     all_counts += counter
# print (albumlist)

In [85]:
def counter_to_vector(counter):
    vector = numpy.zeros(vocabulary_size)
    for word in counter.keys():
        ## look up the integer ID for the string
        word_id = reverse_vocab[word]
#         print (counter[word])
        vector[word_id] = counter[word]
    
    return vector / vector.sum()

In [94]:
albumtokens=defaultdict(dict)
for album in albumlist:
#     print (type(albumlist[album]))
    print (album)
    albumtokens[album]["Tokens"]=Counter()
    for song in albumlist[album]:
        albumtokens[album]["Tokens"].update(song["Tokens"])

channel ORANGE
nostalgia, ULTRA.
Blonde 
Endless


In [100]:
albumtokens


defaultdict(dict,
            {'channel ORANGE': {'Tokens': Counter({'They': 56,
                       'look': 7,
                       'like': 168,
                       'twins': 7,
                       'That': 35,
                       'was': 56,
                       'embarrassing': 7,
                       'A': 63,
                       'tornado': 7,
                       'flew': 7,
                       'around': 49,
                       'my': 469,
                       'room': 21,
                       'before': 21,
                       'you': 903,
                       'came': 14,
                       'Excuse': 7,
                       'the': 1330,
                       'mess': 14,
                       'it': 252,
                       'made': 35,
                       'usually': 7,
                       "doesn't": 7,
                       'rain': 7,
                       'in': 441,
                       'Southern': 7,
                       'Califor

In [87]:
check=Counter()
for i in albumtokens:
    check.update(albumtokens[i]["Tokens"])

In [88]:
vocabulary = [w for w, c in check.most_common()]
# print (vocabulary)
vocabulary_size = len(vocabulary)

reverse_vocab = { w: i for i, w in enumerate(vocabulary) }



{}

In [90]:
all_vector = counter_to_vector(check)

In [12]:

#     print (albumtokens[i]["Tokens"])
print (check)
for album in albumtokens:
#     print (albumtokens[album]["Tokens"]["Yo"])
    albumtokens[album]["Vector"] = counter_to_vector(albumtokens[album]["Tokens"])


Counter({'the': 519, 'I': 507, 'you': 458, 'a': 293, 'to': 279, 'me': 262, 'my': 222, 'in': 194, 'and': 188, 'on': 171, 'it': 132, 'that': 130, 'of': 112, "I'm": 112, 'know': 105, 'for': 100, 'like': 98, 'be': 89, 'your': 89, 'no': 88, 'And': 87, 'You': 86, 'with': 85, 'love': 82, 'do': 82, "don't": 80, 'this': 78, 'is': 75, 'at': 70, 'all': 69, 'we': 66, 'feel': 61, 'was': 60, "can't": 60, 'But': 59, 'when': 58, "it's": 56, 'up': 55, 'good': 55, 'just': 54, 'what': 53, 'got': 51, 'but': 51, 'time': 50, 'not': 50, "It's": 50, 'could': 50, 'The': 49, "ain't": 48, 'see': 46, 'life': 46, 'get': 46, 'out': 44, 'so': 44, 'now': 43, 'night': 42, 'down': 41, 'had': 41, 'shit': 41, 'her': 41, 'We': 40, "you're": 40, 'girl': 37, 'have': 37, 'yeah': 37, 'off': 37, 'been': 36, "You're": 36, 'oh': 35, 'can': 35, 'she': 34, 'from': 34, 'try': 34, 'they': 33, 'say': 33, 'go': 33, 'here': 32, 'back': 32, "I've": 32, 'What': 32, 'believe': 32, 'let': 31, 'think': 31, 'If': 31, 'never': 31, 'run': 31, 

In [13]:
for album in (albumtokens):
#     print 
    # `nonzero` returns a tuple with one element per dimension of the input array
    nzs = numpy.nonzero(albumtokens[album]["Vector"])[0]
#     print (nzs)
    kl_terms = albumtokens[album]["Vector"][nzs] * numpy.log2(albumtokens[album]["Vector"][nzs] / all_vector[nzs])
    
    vocabulary_subset = [vocabulary[i] for i in nzs]

    sorted_words = sorted(zip(kl_terms, vocabulary_subset), reverse=True)
#     print(song_id, song["title"])
    print (album)
    print(", ".join(["{} ({:.3f})".format(w, s) for s, w in sorted_words[:40]]))
    print(", ".join(["{} ({:.3f})".format(w, s) for s, w in sorted_words[-40:]]))
    print()

Blonde 
night (0.010), shit (0.007), solo (0.006), da (0.005), that (0.004), a (0.004), I (0.004), every (0.004), I'd (0.003), It's (0.003), day (0.003), low (0.003), of (0.003), down (0.003), anything (0.003), Momma (0.003), is (0.003), like (0.003), This (0.003), nigga (0.003), showed (0.003), I'll (0.003), not (0.002), leave (0.002), dreamin (0.002), comin (0.002), you (0.002), thought (0.002), we're (0.002), side (0.002), Fuck (0.002), how (0.002), same (0.002), last (0.002), summer (0.002), now (0.002), That's (0.002), up (0.002), this (0.002), start (0.002)
tonight (-0.001), ever (-0.001), em (-0.001), Cause (-0.001), sky (-0.001), world (-0.001), our (-0.001), She (-0.001), feel (-0.001), one (-0.001), baby (-0.001), yeah (-0.001), still (-0.001), we (-0.001), too (-0.001), been (-0.001), are (-0.001), What (-0.001), I've (-0.001), girl (-0.001), never (-0.001), try (-0.001), she (-0.001), can (-0.001), You're (-0.001), But (-0.001), do (-0.001), her (-0.001), get (-0.001), but 